In [ ]:
import sqlite3
import urllib.request
from datetime import datetime, timedelta, timezone
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from PIL import Image
from sgp4.api import Satrec, jday
from sklearn.decomposition import PCA
from scipy.optimize import least_squares


In [ ]:
def compute_satellite_position(row, propagate_time=None):
    tle1 = row["tle_line1"]
    tle2 = row["tle_line2"]
    satrec = Satrec.twoline2rv(tle1, tle2)

    # Use current UTC time if none provided
    if propagate_time is None:
        propagate_time = datetime.now(timezone.utc)

    jd, fr = jday(
        propagate_time.year,
        propagate_time.month,
        propagate_time.day,
        propagate_time.hour,
        propagate_time.minute,
        propagate_time.second + propagate_time.microsecond * 1e-6,
    )

    error_code, r, v = satrec.sgp4(jd, fr)

    if error_code != 0:
        return pd.Series(
            {
                "x": np.nan,
                "y": np.nan,
                "z": np.nan,
                "vx": np.nan,
                "vy": np.nan,
                "vz": np.nan,
            }
        )

    return pd.Series(
        {"x": r[0], "y": r[1], "z": r[2], "vx": v[0], "vy": v[1], "vz": v[2]}
    )

In [ ]:
def plot_satellites_3d(df, cone_size=0.1, cone_scale=0.05):
    # ---- Earth parameters ----
    R_earth = 6371

    # ---- Load Earth texture ----
    img_url = "https://eoimages.gsfc.nasa.gov/images/imagerecords/57000/57730/land_ocean_ice_2048.png"
    with urllib.request.urlopen(img_url) as url:
        img = Image.open(url)
    img = img.resize((1024, 512))
    img_data = np.array(img)

    # ---- Sphere coordinates ----
    u = np.linspace(0, 2 * np.pi, img_data.shape[1])
    v = np.linspace(0, np.pi, img_data.shape[0])
    u, v = np.meshgrid(u, v)
    xe = R_earth * np.cos(u) * np.sin(v)
    ye = R_earth * np.sin(u) * np.sin(v)
    ze = R_earth * np.cos(v)

    # ---- Texture to grayscale for surfacecolor ----
    texture = img_data / 255.0
    surfacecolor = (
        0.299 * texture[:, :, 0] + 0.587 * texture[:, :, 1] + 0.114 * texture[:, :, 2]
    )

    # ---- Earth surface ----
    earth_surface = go.Surface(
        x=xe,
        y=ye,
        z=ze,
        surfacecolor=surfacecolor,
        colorscale="gray",
        cmin=0,
        cmax=1,
        showscale=False,
        hoverinfo="skip",
    )

    # ---- Cone traces for velocities ----
    satellite_cones = go.Cone(
        x=df["x"],
        y=df["y"],
        z=df["z"],
        u=df["vx"] * cone_scale,
        v=df["vy"] * cone_scale,
        w=df["vz"] * cone_scale,
        colorscale=[[0, "red"], [1, "red"]],
        cmin=0,
        cmax=1,
        sizemode="absolute",
        sizeref=cone_size,
        anchor="tail",
        showscale=False,
        name="Starlink",
        text=df["starlink_name"],
        hoverinfo="text",
    )

    # ---- Build figure ----
    fig = go.Figure(data=[earth_surface, satellite_cones])
    fig.update_layout(
        scene=dict(
            xaxis=dict(showbackground=False),
            yaxis=dict(showbackground=False),
            zaxis=dict(showbackground=False),
            aspectmode="data",
        ),
        title="Satellites and Velocities Close to Earth",
    )

    fig.show()

In [ ]:
def plot_satellites_3d_time_slider(
    df,
    minutes_range=60,
    step_minutes=1,
    cone_size=0.1,
    cone_scale=0.05,
):
    """
    Animate satellites over time using cones for velocity vectors, with Earth as a blue sphere.
    """
    R_earth = 6371

    # ---- Prepare Earth sphere ----
    u = np.linspace(0, 2 * np.pi, 50)
    v = np.linspace(0, np.pi, 50)
    x_earth = R_earth * np.outer(np.cos(u), np.sin(v))
    y_earth = R_earth * np.outer(np.sin(u), np.sin(v))
    z_earth = R_earth * np.outer(np.ones_like(u), np.cos(v))

    earth_surface = go.Surface(
        x=x_earth,
        y=y_earth,
        z=z_earth,
        colorscale=[[0, "blue"], [1, "blue"]],
        opacity=0.3,
        showscale=False,
    )

    # ---- Compute global axis ranges to prevent wobble ----
    # Start with Earth min/max
    x_min, x_max = x_earth.min(), x_earth.max()
    y_min, y_max = y_earth.min(), y_earth.max()
    z_min, z_max = z_earth.min(), z_earth.max()

    # Check satellite positions at all timesteps
    times = np.arange(0, minutes_range + 1, step_minutes)
    for t in times:
        df_step = df.assign(
            **df.apply(
                lambda row: compute_satellite_position(
                    row, propagate_time=row["launch_date"] + timedelta(minutes=int(t))
                ),
                axis=1,
            )
        ).dropna(subset=["x", "y", "z"])
        if not df_step.empty:
            x_min = min(x_min, df_step["x"].min())
            x_max = max(x_max, df_step["x"].max())
            y_min = min(y_min, df_step["y"].min())
            y_max = max(y_max, df_step["y"].max())
            z_min = min(z_min, df_step["z"].min())
            z_max = max(z_max, df_step["z"].max())

    # ---- Prepare frames ----
    frames = []

    for t in times:
        df_step = df.assign(
            **df.apply(
                lambda row: compute_satellite_position(
                    row, propagate_time=row["launch_date"] + timedelta(minutes=int(t))
                ),
                axis=1,
            )
        )
        df_step = df_step.dropna(subset=["x", "y", "z", "vx", "vy", "vz"])

        cones = go.Cone(
            x=df_step["x"],
            y=df_step["y"],
            z=df_step["z"],
            u=df_step["vx"] * cone_scale,
            v=df_step["vy"] * cone_scale,
            w=df_step["vz"] * cone_scale,
            colorscale=[[0, "red"], [1, "red"]],
            sizemode="absolute",
            sizeref=cone_size,
            anchor="tail",
            showscale=False,
            text=df_step["starlink_name"],
            hoverinfo="text",
        )

        # Include Earth in every frame so cones animate correctly
        frames.append(go.Frame(data=[earth_surface, cones], name=str(t)))

    # ---- Initial figure ----
    fig = go.Figure(data=frames[0].data, frames=frames)

    # ---- Slider ----
    slider = [
        dict(
            steps=[
                dict(
                    method="animate",
                    args=[
                        [str(t)],
                        dict(
                            mode="immediate",
                            frame=dict(duration=100, redraw=True),
                            transition=dict(duration=0),
                        ),
                    ],
                    label=f"{t} min",
                )
                for t in times
            ],
            currentvalue=dict(prefix="Minutes: "),
            transition=dict(duration=0),
            x=0,
            y=0,
            len=1.0,
        )
    ]

    # ---- Layout with fixed axis ranges ----
    fig.update_layout(
        sliders=slider,
        scene=dict(
            xaxis=dict(showbackground=False, range=[x_min, x_max]),
            yaxis=dict(showbackground=False, range=[y_min, y_max]),
            zaxis=dict(showbackground=False, range=[z_min, z_max]),
            aspectmode="data",
        ),
        title="Starlink Satellites Over Time",
    )

    fig.show()

In [ ]:
con = sqlite3.connect(Path("..", "data", "starlink.db"))
df = pd.read_sql(
    """
                  SELECT 
                    s.id AS starlink_id,
                    s.height_km,
                    s.object_name AS starlink_name, 
                    l.id AS launch_id,
                    l.date AS launch_date,
                    s.tle_line1, 
                    s.tle_line2 
                  FROM starlink s
                  JOIN launch l ON s.launch_id = l.id;
                  
""",
    con,
)

In [ ]:
df["launch_date"] = pd.to_datetime(df["launch_date"])
df.head()

In [ ]:
df = df.assign(
    **df.apply(
        lambda row: compute_satellite_position(row, propagate_time=row["launch_date"]),
        axis=1,
    )
)
df = df.dropna(subset=["x", "y", "z", "vx", "vy", "vz"])
df.head()

In [ ]:
# Drop Satelites that are too far away
R_earth = 6371
df["r"] = np.sqrt(df["x"] ** 2 + df["y"] ** 2 + df["z"] ** 2)
df = df[(df["r"] - R_earth > 100) & (df["r"] - R_earth < 5000)]

In [ ]:
plot_satellites_3d(df[df["launch_id"] == "63161339ffc78f3b8567070c"], cone_size=0.3)

In [ ]:
plot_satellites_3d_time_slider(
    df[df["launch_id"] == "63161339ffc78f3b8567070c"], cone_size=0.3
)

---

# Estimate Orbit

In [ ]:
def fit_ellipse_geometric(points_2d, regularization):
    x = points_2d[:, 0]
    y = points_2d[:, 1]

    # Initial guess: center at mean, axes as std dev, angle 0
    x0_init, y0_init = x.mean(), y.mean()
    a_init, b_init = np.std(x), np.std(y)
    theta_init = 0.0
    params_init = [x0_init, y0_init, a_init, b_init, theta_init]

    def residuals(p):
        x0, y0, a, b, theta = p
        cos_t, sin_t = np.cos(theta), np.sin(theta)
        # Project points to rotated ellipse frame
        dx = x - x0
        dy = y - y0
        xp = cos_t * dx + sin_t * dy
        yp = -sin_t * dx + cos_t * dy
        # Distance to ellipse
        return np.sqrt((xp / a) ** 2 + (yp / b) ** 2) - 1

    if regularization:
        res = least_squares(residuals, params_init, loss="soft_l1", f_scale=0.1)
    else:
        res = least_squares(residuals, params_init)

    return res.x

In [ ]:
def fit_ellipse_params(points_2d):
    x = points_2d[:, 0]
    y = points_2d[:, 1]

    # Construct design matrix for general conic
    D = np.column_stack([x**2, x * y, y**2, x, y, np.ones_like(x)])
    _, _, Vt = np.linalg.svd(D)
    coeffs = Vt[-1, :]
    A, B, C, D_coef, E_coef, F_coef = coeffs

    # Center
    delta = B**2 - 4 * A * C
    x0 = (2 * C * D_coef - B * E_coef) / delta
    y0 = (2 * A * E_coef - B * D_coef) / delta

    # Rotation angle
    theta = 0.5 * np.arctan2(B, A - C)

    # Axes lengths
    up = 2 * (
        A * E_coef**2
        + C * D_coef**2
        + F_coef * B**2
        - 2 * B * D_coef * E_coef
        - A * C * F_coef
    )
    down1 = (B**2 - A * C) * ((C - A) * np.sqrt(1 + (B / (A - C)) ** 2) - (C + A))
    down2 = (B**2 - A * C) * (-(C - A) * np.sqrt(1 + (B / (A - C)) ** 2) - (C + A))
    a = np.sqrt(np.abs(up / down1))
    b = np.sqrt(np.abs(up / down2))

    # Ensure a >= b
    if b > a:
        a, b = b, a
        theta += np.pi / 2

    return x0, y0, a, b, theta

In [ ]:
def plot_2d_ellipsis(points_2d, x0, y0, a, b, theta_rot):
    # Generate parametric ellipse for plotting
    phi = np.linspace(0, 2 * np.pi, 300)
    ellipse_x = (
        x0 + a * np.cos(phi) * np.cos(theta_rot) - b * np.sin(phi) * np.sin(theta_rot)
    )
    ellipse_y = (
        y0 + a * np.cos(phi) * np.sin(theta_rot) + b * np.sin(phi) * np.cos(theta_rot)
    )

    # Plot with Plotly
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=points_2d[:, 0],
            y=points_2d[:, 1],
            mode="markers",
            name="PCA points",
            marker=dict(color="red", size=5),
        )
    )
    fig.add_trace(
        go.Scatter(
            x=ellipse_x,
            y=ellipse_y,
            mode="lines",
            name="Fitted ellipse",
            line=dict(color="blue", width=2),
        )
    )

    fig.update_layout(
        title="Least-Squares Fitted Ellipse",
        xaxis_title="PC1",
        yaxis_title="PC2",
        width=700,
        height=700,
    )
    fig.show()

In [ ]:
df["launch_id"].unique()

In [ ]:
# df_launch = df[df["launch_id"] == "63161339ffc78f3b8567070c"]
df_launch = df[df["launch_id"] == "5ef6a2e70059c33cee4a8293"]
# Select the 3D points
points_3d = df_launch[["x", "y", "z"]].values

# Fit PCA
pca = PCA(n_components=2)
points_2d = pca.fit_transform(points_3d)

In [ ]:
# Create grid in PCA space
x_min, x_max = points_2d[:, 0].min(), points_2d[:, 0].max()
y_min, y_max = points_2d[:, 1].min(), points_2d[:, 1].max()
xx, yy = np.meshgrid(
    np.linspace(x_min, x_max, 20),  # finer grid for smoother surface
    np.linspace(y_min, y_max, 20),
)

# Project grid back to 3D space
grid_points = np.column_stack([xx.ravel(), yy.ravel()])
plane_3d = pca.inverse_transform(grid_points)
X_plane = plane_3d[:, 0].reshape(xx.shape)
Y_plane = plane_3d[:, 1].reshape(xx.shape)
Z_plane = plane_3d[:, 2].reshape(xx.shape)

# Create Plotly figure
fig = go.Figure()

# Original 3D points
fig.add_trace(
    go.Scatter3d(
        x=points_3d[:, 0],
        y=points_3d[:, 1],
        z=points_3d[:, 2],
        mode="markers",
        marker=dict(color="red", size=4),
        name="Original points",
    )
)

# PCA plane
fig.add_trace(
    go.Surface(
        x=X_plane,
        y=Y_plane,
        z=Z_plane,
        colorscale="Blues",
        opacity=0.5,
        name="PCA plane",
        showscale=False,
    )
)

# Layout
fig.update_layout(
    scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z", aspectmode="data"),
    title="3D Points with PCA Plane",
)

fig.show()

In [ ]:
plt.scatter(points_2d[:, 0], points_2d[:, 1])
plt.title("")
plt.xlabel("PCA 1")
plt.ylabel("PCA 2")
plt.show()

In [ ]:
x0, y0, a, b, theta_rot = fit_ellipse_params(points_2d)
plot_2d_ellipsis(points_2d, x0, y0, a, b, theta_rot)

# Estimate Orbit (Data Augmentation)
Computing the position for the satellites

In [ ]:
def augment_df(df, value_space):
    dfs = []
    times = np.arange(0, 90 + 1, value_space)
    for t in times:
        df_step = df.assign(
            **df.apply(
                lambda row: compute_satellite_position(
                    row, propagate_time=row["launch_date"] + timedelta(minutes=int(t))
                ),
                axis=1,
            )
        ).dropna(subset=["x", "y", "z"])

        dfs.append(df_step)

    return pd.concat(dfs, axis=0)

In [ ]:
df_aug = augment_df(df_launch, value_space=5)
# Select the 3D points
points_3d_aug = df_aug[["x", "y", "z"]].values

# Fit PCA
pca = PCA(n_components=2)
points_2d_aug = pca.fit_transform(points_3d_aug)

In [ ]:
plt.scatter(points_2d_aug[:, 0], points_2d_aug[:, 1])
plt.title("")
plt.xlabel("PCA 1")
plt.ylabel("PCA 2")
plt.show()

In [ ]:
x0, y0, a, b, theta_rot = fit_ellipse_geometric(points_2d_aug, regularization=False)
plot_2d_ellipsis(points_2d_aug, x0, y0, a, b, theta_rot)

## Add Regularization

In [ ]:
x0, y0, a, b, theta_rot = fit_ellipse_geometric(points_2d_aug, regularization=True)
plot_2d_ellipsis(points_2d_aug, x0, y0, a, b, theta_rot)

In [ ]:
# --- Step 1: Create ellipse in 2D PCA space ---
phi = np.linspace(0, 2 * np.pi, 300)
ellipse_2d_x = (
    x0 + a * np.cos(phi) * np.cos(theta_rot) - b * np.sin(phi) * np.sin(theta_rot)
)
ellipse_2d_y = (
    y0 + a * np.cos(phi) * np.sin(theta_rot) + b * np.sin(phi) * np.cos(theta_rot)
)
ellipse_2d = np.column_stack([ellipse_2d_x, ellipse_2d_y])

# --- Step 2: Map ellipse back to 3D ---
ellipse_3d = pca.inverse_transform(ellipse_2d)

# --- Step 4: Plot everything in Plotly ---
fig = go.Figure()

# Original 3D points
fig.add_trace(
    go.Scatter3d(
        x=points_3d[:, 0],
        y=points_3d[:, 1],
        z=points_3d[:, 2],
        mode="markers",
        marker=dict(size=3, color="red"),
        name="Original points",
    )
)

# Fitted ellipse in 3D
fig.add_trace(
    go.Scatter3d(
        x=ellipse_3d[:, 0],
        y=ellipse_3d[:, 1],
        z=ellipse_3d[:, 2],
        mode="lines",
        line=dict(color="blue", width=4),
        name="Fitted ellipse",
    )
)

fig.update_layout(
    title="3D PCA Points with Fitted Ellipse",
    scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z", aspectmode="data"),
    width=800,
    height=800,
)

fig.show()

In [ ]:
# --- Step 1: Create ellipse in 2D PCA space ---
phi = np.linspace(0, 2 * np.pi, 300)
ellipse_2d_x = (
    x0 + a * np.cos(phi) * np.cos(theta_rot) - b * np.sin(phi) * np.sin(theta_rot)
)
ellipse_2d_y = (
    y0 + a * np.cos(phi) * np.sin(theta_rot) + b * np.sin(phi) * np.cos(theta_rot)
)
ellipse_2d = np.column_stack([ellipse_2d_x, ellipse_2d_y])

# --- Step 2: Map ellipse back to 3D ---
ellipse_3d = pca.inverse_transform(ellipse_2d)

# --- Step 3: Create Earth surface ---
R_earth = 6371

# Load Earth texture
img_url = "https://eoimages.gsfc.nasa.gov/images/imagerecords/57000/57730/land_ocean_ice_2048.png"
with urllib.request.urlopen(img_url) as url:
    img = Image.open(url)
img = img.resize((1024, 512))
img_data = np.array(img)

# Sphere coordinates
u = np.linspace(0, 2 * np.pi, img_data.shape[1])
v = np.linspace(0, np.pi, img_data.shape[0])
u, v = np.meshgrid(u, v)
xe = R_earth * np.cos(u) * np.sin(v)
ye = R_earth * np.sin(u) * np.sin(v)
ze = R_earth * np.cos(v)

# Convert texture to grayscale for surfacecolor
texture = img_data / 255.0
surfacecolor = (
    0.299 * texture[:, :, 0] + 0.587 * texture[:, :, 1] + 0.114 * texture[:, :, 2]
)

earth_surface = go.Surface(
    x=xe,
    y=ye,
    z=ze,
    surfacecolor=surfacecolor,
    colorscale="gray",
    cmin=0,
    cmax=1,
    showscale=False,
    hoverinfo="skip",
    name="Earth",
)

# --- Step 4: Plot everything ---
fig = go.Figure()

# Add Earth
fig.add_trace(earth_surface)

# Original 3D points
fig.add_trace(
    go.Scatter3d(
        x=points_3d[:, 0],
        y=points_3d[:, 1],
        z=points_3d[:, 2],
        mode="markers",
        marker=dict(size=3, color="red"),
        name="Original points",
    )
)

# Fitted ellipse in 3D
fig.add_trace(
    go.Scatter3d(
        x=ellipse_3d[:, 0],
        y=ellipse_3d[:, 1],
        z=ellipse_3d[:, 2],
        mode="lines",
        line=dict(color="blue", width=4),
        name="Fitted ellipse",
    )
)

fig.update_layout(
    title="3D PCA Points with Fitted Ellipse and Earth",
    scene=dict(
        xaxis_title="X",
        yaxis_title="Y",
        zaxis_title="Z",
        aspectmode="data",
        xaxis=dict(showbackground=False),
        yaxis=dict(showbackground=False),
        zaxis=dict(showbackground=False),
    ),
    width=800,
    height=800,
)

fig.show()

#  Exploration

Orbit Altitude Trend over Time： Analyze how the average orbital altitude of Starlink satellites evolved over time across different launch batches.

Calculate satellite altitude from 3D position

Altitude = distance from Earth's center minus Earth's radius (6371 km)

In [ ]:
R_earth = 6371
df["altitude_km"] = np.sqrt(df["x"] ** 2 + df["y"] ** 2 + df["z"] ** 2) - R_earth
df["launch_date"] = pd.to_datetime(df["launch_date"])

# Compute average altitude for each launch batch
altitude_trend = (
    df.groupby("launch_date")["altitude_km"]
    .mean()
    .reset_index()
    .sort_values("launch_date")
)

# Plot the evolution of average orbit altitude over time
import plotly.express as px

fig = px.line(
    altitude_trend,
    x="launch_date",
    y="altitude_km",
    title="Average Orbit Altitude by Launch Date",
    markers=True,
)
fig.update_traces(line=dict(width=3))
fig.show()

Although the dataset includes all Earth satellites, the sharp changes in orbital altitude over time are primarily driven by SpaceX’s Starlink constellation.
Since 2019, Starlink satellites have dominated the low-Earth orbit population—accounting for over 60% of active satellites worldwide.
The sharp drop in average altitude around mid-2019 reflects the introduction of operational Starlink shells at ~550 km, which reshaped the global LEO distribution pattern.

The early spike in average altitude corresponds to SpaceX’s prototype Starlink batch launched in May 2019 at altitudes above 1,100 km.
In April 2019, SpaceX filed an FCC amendment to lower all operational satellites to 550 km and 53° inclination, which explains the sharp drop in the figure.
Since late 2019, all Starlink satellites have been deployed near 550 km, marking the transition from test to operational constellation.
(Sources: FCC Filing SAT-LOA-20161115-00118, SpaceNews 2019-05, McDowell Starlink Catalog, NASA ODQN 2022.)

Orbital Inclination Distribution：the angle between a satellite’s orbital plane and the Earth’s equatorial plane.
i = 0° → equatorial prograde; i = 90° → polar; i > 90° → retrograde.

In [ ]:
# Compute true inclination from state vectors (r, v)
def true_inclination_from_state(row):
    """
    Return orbital inclination [deg] from position (x,y,z) [km] and velocity (vx,vy,vz) [km/s].
    Uses i = arccos(h_z / |h|) where h = r x v is the specific angular momentum vector.
    """
    r = np.array([row["x"], row["y"], row["z"]], dtype=float)
    v = np.array([row["vx"], row["vy"], row["vz"]], dtype=float)
    h = np.cross(r, v)
    hz = h[2]
    i_rad = np.arccos(hz / np.linalg.norm(h))
    return np.degrees(i_rad)


df["inclination_true"] = df.apply(true_inclination_from_state, axis=1)


df["inclination_true"].describe()


When recalculated using the orbital angular momentum vector (r × v), the distribution collapses into a single dominant peak around 53.
A rule of thumb: the satellite ground track typically reaches up to about ±i in latitude; hence i≈53° shells cover most populated latitudes, while i≈97° (near-polar/Sun-sync) reaches the poles.

In [ ]:
# Bucket by inclination to highlight shells
def bucket_inclination(i):
    if 51 <= i <= 55:
        return "Main shell (~53°)"
    elif 95 <= i <= 99:
        return "Polar shell (~97°)"
    else:
        return "Other"


df["shell_bucket"] = df["inclination_true"].apply(bucket_inclination)

In [ ]:
import plotly.express as px

fig = px.histogram(
    df,
    x="inclination_true",
    color="shell_bucket",
    barmode="overlay",
    nbins=60,
    opacity=0.8,
    title="True Orbital Inclinations (binned by shells)",
    labels={"inclination_true": "Inclination (deg)"},
)
fig.add_vline(x=53, line_dash="dash", line_color="green")
fig.add_vline(x=97, line_dash="dash", line_color="red")
fig.show()

In [ ]:
import seaborn as sns

plt.figure(figsize=(10, 6))

# KDE density plot for orbital inclination
sns.kdeplot(df["inclination_true"], fill=True, color="royalblue", bw_adjust=0.5)

# Add reference lines for known shells
plt.axvline(53, color="green", linestyle="--", label="Main shell (~53°)")
plt.axvline(97, color="red", linestyle="--", label="Polar orbit (~97°)")

plt.title("True Orbital Inclination Density (KDE-based)")
plt.xlabel("Inclination (degrees)")
plt.ylabel("Density")
plt.legend()
plt.grid(alpha=0.3)
plt.show()



The inclination distribution shows a dominant primary mode near 53° (main operational shell) and a minor secondary mode near 97°, corresponding to a small group of near-polar orbits.
Although technically bimodal, the distribution is highly skewed toward the main 53° shell, indicating the majority of satellites occupy a single, tightly controlled inclination layer.
The main shell (~53°) represents the majority of Starlink satellites deployed in mid-inclination orbits to provide near-global coverage between ±53° latitude.
The secondary polar cluster (~97°) includes satellites launched for high-latitude connectivity, extending coverage to polar regions.

In [ ]:
import plotly.graph_objects as go


r_earth = 6371
u, v = np.mgrid[0 : 2 * np.pi : 50j, 0 : np.pi : 25j]
x_sphere = r_earth * np.cos(u) * np.sin(v)
y_sphere = r_earth * np.sin(u) * np.sin(v)
z_sphere = r_earth * np.cos(v)


# Define orbit planes
def orbit_circle(radius, inclination_deg, color, name):
    """Return x, y, z coordinates of an orbit tilted by inclination."""
    theta = np.linspace(0, 2 * np.pi, 200)
    i = np.radians(inclination_deg)
    x = radius * np.cos(theta)
    y = radius * np.sin(theta) * np.cos(i)
    z = radius * np.sin(theta) * np.sin(i)
    return go.Scatter3d(
        x=x,
        y=y,
        z=z,
        mode="lines",
        line=dict(color=color, width=5),
        name=f"{name} (i={inclination_deg}°)",
    )


fig = go.Figure()

# Earth sphere
fig.add_trace(
    go.Surface(
        x=x_sphere,
        y=y_sphere,
        z=z_sphere,
        opacity=0.3,
        colorscale=[[0, "lightblue"], [1, "lightblue"]],
        showscale=False,
        name="Earth",
    )
)

# Reference plane (equatorial)
fig.add_trace(
    go.Surface(
        x=np.outer(np.linspace(-9000, 9000, 2), np.ones(2)),
        y=np.outer(np.linspace(-9000, 9000, 2), np.ones(2)),
        z=np.zeros((2, 2)),
        opacity=0.15,
        colorscale=[[0, "gray"], [1, "gray"]],
        showscale=False,
        name="Equatorial plane",
    )
)

# Main shell orbit (~53°)
fig.add_trace(orbit_circle(9000, 53, "mediumpurple", "Main Starlink orbit"))

# Polar orbit (~97°)
fig.add_trace(orbit_circle(9000, 97, "gold", "Polar orbit"))


fig.update_layout(
    title="Visualization of Orbital Inclination Planes (53° vs 97°)",
    scene=dict(
        xaxis_title="X (km)",
        yaxis_title="Y (km)",
        zaxis_title="Z (km)",
        aspectmode="data",
    ),
    scene_camera=dict(eye=dict(x=1.8, y=1.6, z=1.2)),
    legend=dict(x=0.02, y=0.98, bgcolor="rgba(255,255,255,0.6)"),
)

fig.show()


In [ ]:
# Inclination vs Altitude
R_EARTH_KM = 6371.0
df["altitude_km"] = np.sqrt(df["x"] ** 2 + df["y"] ** 2 + df["z"] ** 2) - R_EARTH_KM

fig = px.scatter(
    df,
    x="inclination_true",
    y="altitude_km",
    color="shell_bucket",
    title="Altitude vs. Inclination (shell structure)",
    labels={"inclination_true": "Inclination (deg)", "altitude_km": "Altitude (km)"},
    opacity=0.7,
)
fig.show()


Different shells tend to cluster at distinct altitudes (layered design)

Velocity vs. Altitude

In [ ]:
# Compute velocity magnitude from vx, vy, vz components
df["velocity_km_s"] = np.sqrt(df["vx"] ** 2 + df["vy"] ** 2 + df["vz"] ** 2)
fig = px.scatter(
    df,
    x="altitude_km",
    y="velocity_km_s",
    title="Satellite Velocity vs. Altitude",
    trendline="ols",
)
fig.show()


The scatterplot demonstrates a clear negative correlation between altitude and orbital velocity.
Satellites at lower altitudes (~550 km) travel faster (~7.6 km/s), while those at higher altitudes move slower (~6 km/s).
This relationship aligns with Kepler’s third law, confirming the physical consistency of the data and model calculations.